<a href="https://colab.research.google.com/github/darkmochalover/DS_TermProject_AL2/blob/main/logistic_regression_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dictionary

In [ ]:
# Data Path
raw_data_path = "/content/drive/MyDrive/DS_TermProject/data/combined_mbti_df.csv"
# data_path = "data/preprocessed_mbti_data.csv"


audio_qualities = [
    'danceability_mean',
    'valence_mean',
    'energy_mean',
    'loudness_mean',
    'acousticness_mean',
    'instrumentalness_mean',
    'liveness_mean',
]

# 장조/단조 (Major/Minor)
all_tones = [
    'Cminor_count', 'CMajor_count', 'C#/Dbminor_count', 'C#/DbMajor_count',
    'DMajor_count', 'D#_EbMajor_count', 'Eminor_count', 'EMajor_count',
    'Fminor_count', 'FMajor_count', 'F#/Gbminor_count', 'GMajor_count',
    'G#/Abminor_count', 'G#/AbMajor_count', 'Aminor_count', 'AMajor_count',
    'A#/Bbminor_count', 'BMajor_count', 'Dminor_count', 'D#_Ebminor_count',
    'Gminor_count', 'A#/BbMajor_count', 'F#/GbMajor_count', 'Bminor_count'
]

major_tones = [
    'CMajor_count', 'C#/DbMajor_count',
    'DMajor_count', 'D#_EbMajor_count', 
    'EMajor_count',
    'FMajor_count', 
    'GMajor_count', 'G#/AbMajor_count', 
    'AMajor_count', 'BMajor_count', 'A#/BbMajor_count', 
    'F#/GbMajor_count'
]
minor_tones = [
    'Cminor_count', 'C#/Dbminor_count', 
    'Eminor_count', 
    'Fminor_count', 'F#/Gbminor_count', 
    'G#/Abminor_count',  
    'Aminor_count', 'A#/Bbminor_count', 
    'Dminor_count', 'D#_Ebminor_count',
    'Gminor_count', 
    'Bminor_count'
]


# 열 이름을 바꾸기 위해 리스트로 저장함
renamed_columns =  [
    'danceability',
    'valence',
    'energy',
    'loudness',
    'acousticness',
    'instrumentalness',
    'liveness'
]


Install WandB

In [ ]:
!pip install -qU wandb

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from imblearn.over_sampling import SMOTE

import itertools

import wandb
import random

Step 1: Import W&B and Login

In [ ]:
import wandb
from wandb.keras import WandbMetricsLogger, WandbModelCheckpoint, WandbEvalCallback

In [ ]:
encoder_list = ['LabelEncoder','OneHotEncoder']
scaler_list = ['StandardScaler', 'MinMaxScaler', 'RobustScaler', 'No Scale']
test_size_list = [0.3, 0.2, 0.1]
do_smote = ['True', 'False']

In [ ]:
config = {
        'encoder' : 'LabelEncoder',
        'scaler' : 'StandardScaler',
        'test_size' : 0.3,
        "do_smote" : True
    }

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="regression",
    
    # track hyperparameters and run metadata
    config = config
)

config=wandb.config


Data Loading

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd

raw_data_path = '/content/drive/MyDrive/DS_TermProject/data/combined_mbti_df.csv'
df = pd.read_csv(raw_data_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Feature Engineering

In [ ]:
# Subset only measures of centers
X = df.iloc[: , :22] # 처음 22개 열을 선택해서 저장
X = df[audio_qualities] # 오디오 품질과 해당 열이 있는 열을 선택해서 저장


categories = renamed_columns[:]
X.columns = renamed_columns


# 장조/단조의 개수의 합을 계산해서 저장 (C장조, D단조, .. 이렇게 따로 계산되는거 말고, 위에 지정된 list 이용해서 sum값 넣어줌)
X['major_count'] = df[major_tones].sum(axis=1).astype('int64')
X['minor_count'] = df[minor_tones].sum(axis=1).astype('int64')

<ipython-input-26-482ca4c8bc7c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['major_count'] = df[major_tones].sum(axis=1).astype('int64')
<ipython-input-26-482ca4c8bc7c>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['minor_count'] = df[minor_tones].sum(axis=1).astype('int64')


## Encoding & Scaling Function

In [ ]:
def Encoding(df, encoding_method):
    df = df.copy()

    if(encoding_method == 'LabelEncoder'):
        encoder = LabelEncoder()
        target = encoder.fit_transform(df[['mbti']])
        

    if(encoding_method == 'OneHotEncoder'):
        encoder = OneHotEncoder(sparse=False)
        target = encoder.fit_transform(df[['mbti']])

    return target

def Scaling(scale_method, X_train, X_test):
    if( scale_method == 'No Scale'):
        return X_train, X_test

    elif(scale_method == 'StandardScaler'):
        scaler = StandardScaler()

    elif(scale_method == 'MinMaxScaler'):
        scaler = MinMaxScaler()

    elif(scale_method == 'RobustScaler'):
        scaler = RobustScaler()


    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.fit_transform(X_test)

    return X_train_scaled, X_test_scaled


In [ ]:
# If X and y have different shapes, adjust them accordingly
if X.shape[0] > y.shape[0]:
    X = X[:y.shape[0], :]
else:
    y = y[:X.shape[0]]

# Print the adjusted shape of X and y
print("Adjusted X shape:", X.shape)
print("Adjusted y shape:", y.shape)

Adjusted X shape: (4081, 9)
Adjusted y shape: (4081,)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, recall_score, precision_score
import itertools
import wandb
import numpy as np

# Grid search hyperparameters
param_grid = {
    'penalty': ['l1', 'l2'],
    'C': [0.1, 1.0, 10.0]
}

# Define the combinations of hyperparameters
encoder_list = ['LabelEncoder', 'OneHotEncoder']
scaler_list = ['StandardScaler', 'MinMaxScaler', 'RobustScaler', 'No Scale']
test_size_list = [0.3, 0.2, 0.1]
do_smote_list = [True, False]

combinations = list(itertools.product(encoder_list, scaler_list, test_size_list, do_smote_list))

for encoder, scaler, test_size, do_smote in combinations:
    # Print the hyperparameter settings
    print(f"Encoder: {encoder}, Scaler: {scaler}, Test Size: {test_size}, SMOTE: {do_smote}")
    wandb.init(project='logistic_regression', config={
        'encoder': encoder,
        'scaler': scaler,
        'test_size': test_size,
        'do_smote': do_smote
    })

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=5, test_size=test_size)

    # Apply the specified encoder and scaler
    if encoder == 'LabelEncoder':
        encoder = LabelEncoder()
        y_train = encoder.fit_transform(y_train)
        y_test = encoder.transform(y_test)
    elif encoder == 'OneHotEncoder':
        encoder = OneHotEncoder()
        y_train = encoder.fit_transform(y_train.reshape(-1, 1)).toarray()
        y_test = encoder.transform(y_test.reshape(-1, 1)).toarray()

    if scaler == 'StandardScaler':
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    elif scaler == 'MinMaxScaler':
        scaler = MinMaxScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    elif scaler == 'RobustScaler':
        scaler = RobustScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    elif scaler == 'No Scale':
        # No scaling
        pass

    # Perform SMOTE if enabled
    if do_smote:
        smote = SMOTE(random_state=42)
        X_train, y_train = smote.fit_resample(X_train, y_train)

    # Create and fit the logistic regression model
    model = LogisticRegression(random_state=42)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    predictions = model.predict(X_test)

    # Calculate evaluation metrics
    accuracy = accuracy_score(y_test, predictions)
    cm = confusion_matrix(y_test, predictions)
    f1 = f1_score(y_test, predictions, average='macro')
    recall = recall_score(y_test, predictions, average='macro')
    precision = precision_score(y_test, predictions, average='macro')

    # Log the evaluation metrics to WandB
    wandb.log({'Accuracy': accuracy})
    wandb.log({'Confusion Matrix': cm})
    wandb.log({'F1 Score': f1})
    wandb.log({'Recall': recall})
    wandb.log({'Precision': precision})

    # Print the evaluation metrics
    print("Accuracy:", accuracy)
    print("Confusion Matrix:\n", cm)
    print("F1 Score:", f1)
    print("Recall:", recall)
    print("Precision:", precision)


Encoder: LabelEncoder, Scaler: StandardScaler, Test Size: 0.3, SMOTE: True


Accuracy: 0.21551020408163266
Confusion Matrix:
 [[12 10  4  1 12  2  5  3  0  5  3  5  6  4  2  4]
 [ 3 25  0  6 11 10  7  2  2  2  3  1  4  3  2  2]
 [ 2  1 26 11  5  1 12  9  2  0  9  1  1  0  0  6]
 [ 2  4  4 37  3  8 11  9  1  1  3  3  0  0  1  7]
 [ 1  7  0  2  5  1  2  1  1  3  2  0  5  0  1  1]
 [ 1 12  5  2 10 14  3  9  1  2  2  1  5  0  1  0]
 [ 1  5  7  3  2  2  7  3  0  0  0  3  3  1  3  1]
 [ 3  5 13 13  7 12  6 19  1  0  9  2  2  2  0  5]
 [ 6  5  0  1  2  0  2  0 17 13 13  7 20  2  5  3]
 [ 9  5  1  3  1  1  4  2 15 13  5  6 16  1  4  3]
 [ 3  3  9  9  2  0  3  2  7  1 22 18  4  0  9  2]
 [ 6  6  5  4  3  2  3  3  2  3  8 25  5  4  6  5]
 [ 3  7  0  0  4  1  1  1  9  6  2  4 18  3  2  2]
 [ 1  3  3  2  5  3  7  0  6  4 10  2  9  7  3  5]
 [10  1  4  0  6  1  2  0  5  4  2  6  7  2  3  6]
 [ 0  8 10 11  2  3  6  3  2  2  8  5  4  1  4 14]]
F1 Score: 0.20181350905402895
Recall: 0.2072490218838857
Precision: 0.21017680242879336
Encoder: LabelEncoder, Scaler: StandardScaler,

Accuracy,▁
F1 Score,▁
Precision,▁
Recall,▁
Accuracy,0.21551
F1 Score,0.20181
Precision,0.21018
Recall,0.20725


Accuracy: 0.21959183673469387
Confusion Matrix:
 [[19 15  6  1  0  3  0  3  0  6  3  4  5  8  1  4]
 [ 7 31  3  8  0 11  0  5  2  2  4  1  2  5  0  2]
 [ 4  1 35 14  0  3  0  8  2  1  8  0  0  2  0  8]
 [ 3  7 12 40  0  6  0 11  1  1  3  4  0  0  0  6]
 [ 4 10  2  2  1  0  0  1  1  2  2  0  4  2  0  1]
 [ 5 20  7  3  1  7  0 12  1  2  2  1  4  3  0  0]
 [ 2  8 12  3  0  0  0  5  2  1  1  3  1  2  0  1]
 [ 6 10 17 17  0  7  0 19  1  0  9  2  2  2  0  7]
 [ 9  6  0  1  0  1  0  0 21 13 15  7 14  6  0  3]
 [14  8  3  3  0  1  1  2 16 11  5  6 11  5  0  3]
 [ 5  5 10 10  0  0  0  2  7  1 27 18  2  5  0  2]
 [11  8  6  2  0  0  0  4  3  4 10 24  2  6  2  8]
 [10  9  0  0  0  1  0  1  9  9  3  5  9  4  2  1]
 [ 5  3  7  4  0  2  0  0  8  5 11  3  7 10  0  5]
 [18  1  6  0  0  1  0  0  5  4  2  6  6  4  0  6]
 [ 5 11 16 10  0  1  0  3  1  3  9  4  2  3  0 15]]
F1 Score: 0.18237838114177587
Recall: 0.19614913369257425
Precision: 0.20452633042209173
Encoder: LabelEncoder, Scaler: StandardScaler

Accuracy,▁
F1 Score,▁
Precision,▁
Recall,▁
Accuracy,0.21959
F1 Score,0.18238
Precision,0.20453
Recall,0.19615


...

Accuracy,▁
F1 Score,▁
Precision,▁
Recall,▁
Accuracy,0.13447
F1 Score,0.11403
Precision,0.141
Recall,0.12378


Accuracy: 0.1466992665036675
Confusion Matrix:
 [[ 2  5  1  1  0  2  0  2  1  0  1  0  3  0  0  0]
 [ 0  9  1  4  0  2  0  1  2  0  1  0  1  0  0  1]
 [ 0  2  9  7  0  0  0  6  1  0  3  0  0  0  1  0]
 [ 0  2  5  8  0  1  0  3  0  1  2  0  0  0  0  2]
 [ 1  4  0  0  0  2  0  2  0  0  0  0  2  0  0  0]
 [ 4  8  3  1  0  1  0  6  0  0  1  0  1  0  0  1]
 [ 1  4  4  1  0  0  0  1  0  0  1  2  1  0  0  0]
 [ 3  6  7  5  0  4  0  6  0  0  2  1  0  0  0  1]
 [ 3  1  0  0  0  0  0  1  9  6  7  0  1  0  1  0]
 [ 2  3  0  1  0  0  0  2  9  6  3  1  1  0  1  0]
 [ 2  5  6  2  0  0  0  1  3  0  2  3  0  0  0  3]
 [ 6  3  3  2  0  0  0  3  6  2  4  3  3  0  1  4]
 [ 6  3  0  1  0  0  0  0  2  5  5  2  5  0  0  0]
 [ 4  2  4  1  0  1  0  1  3  2 10  0  3  0  0  2]
 [ 4  2  3  0  0  0  0  0  4  2  2  2  1  0  0  0]
 [ 2  2  6  5  0  1  0  2  2  0  2  0  0  0  0  0]]
F1 Score: 0.11378815088186447
Recall: 0.1382812210937211
Precision: 0.10991620388225876
Encoder: OneHotEncoder, Scaler: StandardScaler,

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy,▁
F1 Score,▁
Precision,▁
Recall,▁
Accuracy,0.1467
F1 Score,0.11379
Precision,0.10992
Recall,0.13828


ValueError: ignored

In [ ]:
wandb.finish()